## 이미지 썸네일로부터 색상정보 추출
### extcolors 모듈 사용

In [ ]:
%pip install pillow

In [ ]:
%pip install extcolors python-dotenv pymysql

In [ ]:
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
import extcolors

### csv 파일에서 URL을 읽을 때

In [ ]:
# CSV 파일 로드
csv_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\videos_with_captions_full_0312_0322.csv"
df = pd.read_csv(csv_path)

def extract_colors_from_url(url, tolerance=12, limit=5):
    try:
        # 이미지 다운로드
        response = requests.get(url)
        response.raise_for_status()  # 오류 발생 시 예외 발생
        img = Image.open(BytesIO(response.content))
        
        # extcolors로 색상 추출
        # colors: [((R, G, B), 픽셀수), ...], pixel_count: 사용된 총 픽셀수
        colors, pixel_count = extcolors.extract_from_image(img, tolerance=tolerance, limit=limit)
        
        # 리스트에 포함된 색상의 총 픽셀 수 합산
        total = sum(count for (rgb, count) in colors)
        
        # 각 색상의 RGB, 픽셀 수, 그리고 전체 대비 퍼센트 계산
        colors_with_percentage = [(rgb, count, round((count / total * 100), 2)) for (rgb, count) in colors]
        return colors_with_percentage
    except Exception as e:
        print(f"URL 처리 중 오류 발생: {url}\n오류 메시지: {e}")
        return None

# thumbnailURL 열의 각 URL에서 색상 추출 후 새로운 열에 저장
df['extracted_colors'] = df['thumbnailURL'].apply(extract_colors_from_url)

# 결과 CSV 저장
output_csv_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\videos_with_color_captions_0312_0322.csv"
df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print("모든 이미지의 색상 추출이 완료되었습니다.")

### DB에서 URL을 읽을 때

In [ ]:
from dotenv import load_dotenv
import os

# .env 파일의 환경 변수 로드
load_dotenv()

# 환경 변수에서 DB 암호 가져오기 (여기서는 변수명이 '1P_DB_PW'라고 가정)
db_password = os.getenv('1P_DB_PW')
print(db_password)

In [ ]:
import os
import requests
from io import BytesIO
from PIL import Image
import extcolors
import mysql.connector
import json

# DB 접속 설정
db_config = {
    'host': '121.128.172.79',
    'user': 'user3',
    'password': db_password,
    'database': 'yttest_db',
    'port': 3306
}

# DB 연결 생성 (dictionary=True 옵션으로 컬럼명을 key로 사용)
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)

def extract_colors_from_url(url, tolerance=12, limit=5):
    try:
        # 이미지 다운로드 및 열기
        response = requests.get(url)
        response.raise_for_status()  # 오류 발생 시 예외 처리
        img = Image.open(BytesIO(response.content))
        
        # 이미지에서 색상 추출: colors는 [((R, G, B), 픽셀수), ...]
        colors, pixel_count = extcolors.extract_from_image(img, tolerance=tolerance, limit=limit)
        total = sum(count for (rgb, count) in colors)
        
        # 각 색상의 RGB, 픽셀 수, 그리고 전체 대비 퍼센트 계산
        colors_with_percentage = [
            (rgb, count, round((count / total * 100), 2)) for (rgb, count) in colors
        ]
        return colors_with_percentage
    except Exception as e:
        print(f"URL 처리 중 오류 발생: {url}\n오류 메시지: {e}")
        return None

# 1. DB에서 thumbnailURL 컬럼을 읽어오기 (필요한 경우 추가 조건을 붙일 수 있음)
select_query = "SELECT thumbnailURL FROM thumbnail"
cursor.execute(select_query)
rows = cursor.fetchall()

# 2. 각 thumbnailURL에 대해 색상 추출 후 DB 업데이트 수행
for row in rows:
    url = row['thumbnailURL']
    colors = extract_colors_from_url(url)
    if colors is not None:
        # 추출한 색상 정보를 JSON 문자열로 변환
        colors_json = json.dumps(colors)
    else:
        colors_json = None

    # thumbnail 테이블의 colorpalette 컬럼에 업데이트 (thumbnailURL을 PK로 사용)
    update_query = """
        UPDATE thumbnail
        SET colorpalette = %s
        WHERE thumbnailURL = %s
    """
    cursor.execute(update_query, (colors_json, url))
    conn.commit()  # 각 업데이트 후 커밋 (많은 데이터라면 배치 커밋 고려)

print("모든 이미지의 색상 추출 및 DB 업데이트가 완료되었습니다.")

cursor.close()
conn.close()